# 0.0. IMPORTS

In [36]:
import pandas as pd
import pandas.io.sql as psql
import psycopg2 as pg

## 0.1. Helper Functions

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
jupyter_settings() 

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## 0.2. Loading Data

In [2]:
conn2 = pg.connect( dbname = "comunidadedsdb",
                  user = "member",
                  password = "cdspa",
                  host = "comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com",
                  port = "5432")

In [3]:
sql = """ SELECT *
        FROM pa004.users u
            LEFT JOIN pa004.vehicle v ON u.id = v.id 
            LEFT JOIN pa004.insurance i ON u.id = i.id """

In [4]:
df_raw = psql.read_sql_query( sql, conn2 )

C:\repos\anaconda3\envs\insurance_all\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


DatabaseError: Execution failed on sql:  SELECT *
        FROM pa004.users u
            LEFT JOIN pa004.vehicle v ON u.id = v.id 
            LEFT JOIN pa004.insurance i ON u.id = i.id 
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

unable to rollback

In [ ]:
# Fechando conexão
conn2.close()

In [27]:
!cd

C:\Users\Leonardo de Freitas\repos\Cross-Sell-Insurance-All


In [28]:
# Exportando dados
df_raw.to_csv('df_raw.csv', index=False, sep=';')

In [ ]:
df_raw.head()

In [20]:
df_raw.shape

(381109, 14)

# PA004 - Live 015 [1:10:08] Conectando usando o Python

In [41]:
# Credentials

host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port = 5432
database = 'comunidadedsdb'
username = 'member'
pwd = 'cdspa'

In [42]:
conn = pg.connect( user=username,
                   password=pwd,
                   host=host,
                   port=port,
                   database=database )

## 1.0 - Schema Query

In [29]:
cursor = conn.cursor()

query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace

"""

cursor.execute( query_schema )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()

[('pg_toast',), ('pg_temp_1',), ('pg_toast_temp_1',), ('pg_catalog',), ('information_schema',), ('public',), ('pa004',), ('pg_temp_4',), ('pg_toast_temp_4',), ('pa005',), ('pg_temp_9',), ('pg_toast_temp_9',)]


## 2.0 - Table Query

In [26]:
cursor = conn.cursor()

query_tables = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname= 'pa004'

"""

cursor.execute( query_tables )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()

[('users',), ('vehicle',), ('insurance',)]


## 3.0 - Collect Data using Psycopg2

In [31]:
cursor = conn.cursor()

query_tables_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10

"""

cursor.execute( query_tables_users )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()

[(2, 'Male', 76, 3.0, 26.0), (3, 'Male', 47, 28.0, 26.0), (8, 'Female', 56, 28.0, 26.0), (11, 'Female', 47, 35.0, 124.0), (14, 'Male', 76, 28.0, 13.0), (15, 'Male', 71, 28.0, 30.0), (20, 'Female', 60, 33.0, 124.0), (21, 'Male', 65, 28.0, 124.0), (22, 'Male', 49, 28.0, 124.0), (27, 'Female', 51, 28.0, 124.0)]


In [33]:
data = pd.DataFrame( record )
data.head()

,0,1,2,3,4
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


## 4.0 - Collect Data using Pandas

In [38]:
cursor = conn.cursor()

query_tables_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10

"""

df = psql.read_sql( query_tables_users, conn )

df.head()

C:\repos\anaconda3\envs\insurance_all\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,gender,age,region_code,policy_sales_channel
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


In [39]:
df1 = pd.read_sql( query_tables_users, conn )

df1.head()

C:\repos\anaconda3\envs\insurance_all\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,gender,age,region_code,policy_sales_channel
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


In [47]:
cursor = conn.cursor()

query_tables_users = """
    SELECT *
    FROM pa004.users u LEFT JOIN pa004.vehicle v ON ( u.id = v.id )
                       LEFT JOIN pa004.insurance i ON ( u.id = i.id )
    WHERE u.age > 44
    LIMIT 10
    

"""

df = psql.read_sql( query_tables_users, conn )
df = df.drop( df.columns[[5, 9]], axis=1 )
df.head()

C:\repos\anaconda3\envs\insurance_all\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Male,67,18.0,124.0,1,1-2 Year,Yes,0,40277.0,106,0
1,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0
2,Male,50,30.0,26.0,1,1-2 Year,Yes,0,34271.0,229,1
3,Male,49,28.0,124.0,1,1-2 Year,Yes,0,2630.0,88,0
4,Male,50,28.0,26.0,1,1-2 Year,Yes,0,57389.0,155,1


# 1.0. STEP 01 - DATA DESCRIPTION

## 1.1. Rename Columns

## 1.2. Data Dimensions

## 1.3. Data Types 

## 1.4. Check NA 

## 1.5. Fillout NA 

## 1.6. Change Types

## 1.7. Descriptive Statistics

### 1.7.1. Numerical Attributes

### 1.7.2. Categorical Attributes

# 2.0. STEP 02 - FEATURING ENGINEERING

In [23]:
df2 = df1.copy()

## 2.1. Mind Map Hypotheses

## 2.2. Hypotheses Creation

## 2.3 Final List of Hypothesis

## 2.4. Feature Engineering

# 3.0. STEP 03 - FILTERING VARIABLES

In [30]:
df3 = df2.copy()

## 3.1. Rows Filtering

## 3.2. Columns Selection

# 4.0. STEP 04 - EXPLORATORY DATA ANALYSIS

In [37]:
df4 = df3.copy()

## 4.1. Univariate Analysis

### 4.1.1. Response Variable

### 4.1.2. Numerical Variables

### 4.1.3. Categorical Variables

## 4.2. Bivariate Analysis

### 4.2.1. Hypotheses Summary

## 4.3. Multivariate Analysis

### 4.3.1. Numerical Attributes

### 4.3.2. Categorical Attributes

# 5.0. STEP 05 - DATA PREPARATION

In [62]:
df5 = df4.copy()

## 5.1. Normalization

## 5.2. Rescaling

## 5.3. Transformation

### 5.3.1. Encoding

### 5.3.2. Response Variable Transformation

In [65]:
df5['sales'] = np.log1p( df5['sales'] )

# 6.0. STEP 06 - FEATURE SELECTION

In [70]:
df6 = df5.copy()

## 6.1. Split dataframe into training and test dataset

## 6.2. Boruta as Feature Selector

### 6.2.1. Best Features from Boruta

## 6.2. Manual Feature Selection

# 7.0. STEP 07 - MACHINE LEARNING MODELING

## 7.1. Average Model

## 7.2. Linear Regression Model

### 7.2.1. Linear Regression Model - Cross Validation

## 7.3. Linear Regression Regularized Model - Lasso

### 7.3.1. Linear Regression Regularized Model - Lasso - Cross Validation

## 7.4. Random Forest Regressor

### 7.4.1. Random Forest Regressor Model - Cross Validation

## 7.5. XGBoost Regressor

### 7.5.1. XGBoost Regressor Model - Cross Validation

## 7.6. Compare Models Performance

## 7.6.1. Single Performance

## 7.6.2. Real Performance - Cross Validation

# 8.0. STEP 08 - HYPERPARAMETER FINE TUNING

## 8.1. Random Search

## 8.2. Final Model

# 9.0. STEP 09 - PERFORMANCE EVALUATION

## 9.1. Business Performance

## 9.2. Total Performance

## 9.3. Machine Learning Performance

# 10.0. STEP 10 - MODEL DEPLOYMENT

## 10.1. Rossmann Class

## 10.2. API Handler

## 10.3. API Tester